## ಎम्बೆಡ್ಡಿಂಗ್ಸ್

ನಮ್ಮ ಹಿಂದಿನ ಉದಾಹರಣೆಯಲ್ಲಿ, ನಾವು `vocab_size` ಉದ್ದದ ಉನ್ನತ-ಆಯಾಮದ ಬ್ಯಾಗ್-ಆಫ್-ವರ್ಡ್ಸ್ ವೆಕ್ಟರ್‌ಗಳ ಮೇಲೆ ಕಾರ್ಯನಿರ್ವಹಿಸಿದ್ದೇವೆ, ಮತ್ತು ನಾವು ಕಡಿಮೆ-ಆಯಾಮದ ಸ್ಥಾನಿಕ ಪ್ರತಿನಿಧಿ ವೆಕ್ಟರ್‌ಗಳನ್ನು ಸ್ಪಾರ್ಸ್ ಒನ್-ಹಾಟ್ ಪ್ರತಿನಿಧಿಗೆ ಸ್ಪಷ್ಟವಾಗಿ ಪರಿವರ್ತಿಸಿದ್ದೇವೆ. ಈ ಒನ್-ಹಾಟ್ ಪ್ರತಿನಿಧಿ ಸ್ಮೃತಿ-ಕಾರ್ಯಕ್ಷಮವಲ್ಲ. ಜೊತೆಗೆ, ಪ್ರತಿ ಪದವನ್ನು ಪರಸ್ಪರ ಸ್ವತಂತ್ರವಾಗಿ ಪರಿಗಣಿಸಲಾಗುತ್ತದೆ, ಆದ್ದರಿಂದ ಒನ್-ಹಾಟ್ ಎನ್‌ಕೋಡ್ ಮಾಡಿದ ವೆಕ್ಟರ್‌ಗಳು ಪದಗಳSemantic ಸಮಾನತೆಗಳನ್ನು ವ್ಯಕ್ತಪಡಿಸುವುದಿಲ್ಲ.

ಈ ಘಟಕದಲ್ಲಿ, ನಾವು **News AG** ಡೇಟಾಸೆಟ್ ಅನ್ನು ಮುಂದುವರೆಸಿ ಅನ್ವೇಷಿಸುವೆವು. ಪ್ರಾರಂಭಿಸಲು, ಡೇಟಾವನ್ನು ಲೋಡ್ ಮಾಡಿ ಮತ್ತು ಹಿಂದಿನ ಘಟಕದಿಂದ ಕೆಲವು ವ್ಯಾಖ್ಯಾನಗಳನ್ನು ಪಡೆಯೋಣ.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### ಎम्बೆಡ್ಡಿಂಗ್ ಎಂದರೆ ಏನು?

**ಎಂಬೆಡ್ಡಿಂಗ್** ಎಂಬ ಕಲ್ಪನೆ ಎಂದರೆ ಪದಗಳನ್ನು ಕಡಿಮೆ ಆಯಾಮದ ಸಾಂದ್ರ ವೆಕ್ಟರ್‌ಗಳ ಮೂಲಕ ಪ್ರತಿನಿಧಿಸುವುದು, ಅವು ಪದದ ಅರ್ಥವನ್ನು ಪ್ರತಿಬಿಂಬಿಸುತ್ತವೆ. ನಾವು ನಂತರ ಅರ್ಥಪೂರ್ಣ ಪದ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ಹೇಗೆ ನಿರ್ಮಿಸುವುದನ್ನು ಚರ್ಚಿಸುವೆವು, ಆದರೆ ಈಗಾಗಲೇ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ಪದ ವೆಕ್ಟರ್‌ನ ಆಯಾಮವನ್ನು ಕಡಿಮೆ ಮಾಡುವ ವಿಧಾನವೆಂದು ಪರಿಗಣಿಸೋಣ.

ಹೀಗಾಗಿ, ಒಂದು ಎम्बೆಡ್ಡಿಂಗ್ ಲೇಯರ್ ಒಂದು ಪದವನ್ನು ಇನ್‌ಪುಟ್ ಆಗಿ ತೆಗೆದುಕೊಳ್ಳುತ್ತದೆ ಮತ್ತು ನಿರ್ದಿಷ್ಟ `embedding_size` ಹೊಂದಿರುವ ಔಟ್‌ಪುಟ್ ವೆಕ್ಟರ್ ಅನ್ನು ಉತ್ಪಾದಿಸುತ್ತದೆ. ಒಂದು ಅರ್ಥದಲ್ಲಿ, ಇದು `Dense` ಲೇಯರ್‌ಗೆ ಬಹಳ ಸಮಾನವಾಗಿದೆ, ಆದರೆ ಒಂದು-ಹಾಟ್ ಎನ್‌ಕೋಡ್ ಮಾಡಿದ ವೆಕ್ಟರ್ ಅನ್ನು ಇನ್‌ಪುಟ್ ಆಗಿ ತೆಗೆದುಕೊಳ್ಳುವ ಬದಲು, ಇದು ಪದ ಸಂಖ್ಯೆಯನ್ನು ತೆಗೆದುಕೊಳ್ಳಲು ಸಾಧ್ಯವಾಗುತ್ತದೆ.

ನಮ್ಮ ನೆಟ್‌ವರ್ಕ್‌ನಲ್ಲಿ ಮೊದಲ ಲೇಯರ್ ಆಗಿ ಎम्बೆಡ್ಡಿಂಗ್ ಲೇಯರ್ ಅನ್ನು ಬಳಸುವುದರಿಂದ, ನಾವು ಬ್ಯಾಗ್-ಆಫ್-ವರ್ಡ್ಸ್‌ನಿಂದ **ಎಂಬೆಡ್ಡಿಂಗ್ ಬ್ಯಾಗ್** ಮಾದರಿಯ ಕಡೆಗೆ ಬದಲಾಗಬಹುದು, ಅಲ್ಲಿ ನಾವು ಮೊದಲಿಗೆ ನಮ್ಮ ಪಠ್ಯದ ಪ್ರತಿಯೊಂದು ಪದವನ್ನು ಅದರ ಸಂಬಂಧಿಸಿದ ಎम्बೆಡ್ಡಿಂಗ್‌ಗೆ ಪರಿವರ್ತಿಸುತ್ತೇವೆ, ನಂತರ ಆ ಎಲ್ಲಾ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳ ಮೇಲೆ `sum`, `average` ಅಥವಾ `max` ಮುಂತಾದ ಸಂಗ್ರಹಣಾ ಕಾರ್ಯವನ್ನು ಲೆಕ್ಕಹಾಕುತ್ತೇವೆ.

![ಐದು ಕ್ರಮ ಪದಗಳಿಗಾಗಿ ಎम्बೆಡ್ಡಿಂಗ್ ವರ್ಗೀಕರಣವನ್ನು ತೋರಿಸುವ ಚಿತ್ರ.](../../../../../translated_images/kn/embedding-classifier-example.b77f021a7ee67eee.webp)

ನಮ್ಮ ವರ್ಗೀಕರಣ ನ್ಯೂರಲ್ ನೆಟ್‌ವರ್ಕ್ ಕೆಳಗಿನ ಲೇಯರ್‌ಗಳಿಂದ ಕೂಡಿದೆ:

* `TextVectorization` ಲೇಯರ್, ಇದು ಸ್ಟ್ರಿಂಗ್ ಅನ್ನು ಇನ್‌ಪುಟ್ ಆಗಿ ತೆಗೆದುಕೊಳ್ಳುತ್ತದೆ ಮತ್ತು ಟೋಕನ್ ಸಂಖ್ಯೆಗಳ ಟೆನ್ಸರ್ ಅನ್ನು ಉತ್ಪಾದಿಸುತ್ತದೆ. ನಾವು ಕೆಲವು ಯುಕ್ತಿಪೂರ್ಣ ಶಬ್ದಕೋಶ ಗಾತ್ರ `vocab_size` ಅನ್ನು ನಿರ್ಧರಿಸುವೆವು ಮತ್ತು ಕಡಿಮೆ ಬಳಕೆಯ ಪದಗಳನ್ನು ನಿರ್ಲಕ್ಷಿಸುವೆವು. ಇನ್‌ಪುಟ್ ಆಕಾರ 1 ಆಗಿರುತ್ತದೆ ಮತ್ತು ಔಟ್‌ಪುಟ್ ಆಕಾರ $n$ ಆಗಿರುತ್ತದೆ, ಏಕೆಂದರೆ ನಾವು $n$ ಟೋಕನ್‌ಗಳನ್ನು ಫಲಿತಾಂಶವಾಗಿ ಪಡೆಯುತ್ತೇವೆ, ಪ್ರತಿಯೊಂದು 0 ರಿಂದ `vocab_size` ವರೆಗೆ ಸಂಖ್ಯೆಗಳನ್ನೊಳಗೊಂಡಿರುತ್ತದೆ.
* `Embedding` ಲೇಯರ್, ಇದು $n$ ಸಂಖ್ಯೆಗಳನ್ನ ತೆಗೆದುಕೊಳ್ಳುತ್ತದೆ ಮತ್ತು ಪ್ರತಿಯೊಂದು ಸಂಖ್ಯೆಯನ್ನು ನಿರ್ದಿಷ್ಟ ಉದ್ದದ (ನಮ್ಮ ಉದಾಹರಣೆಯಲ್ಲಿ 100) ಸಾಂದ್ರ ವೆಕ್ಟರ್‌ಗೆ ಕಡಿಮೆ ಮಾಡುತ್ತದೆ. ಹೀಗಾಗಿ, $n$ ಆಕಾರದ ಇನ್‌ಪುಟ್ ಟೆನ್ಸರ್ ಅನ್ನು $n\times 100$ ಆಕಾರದ ಟೆನ್ಸರ್‌ಗೆ ಪರಿವರ್ತಿಸಲಾಗುತ್ತದೆ.
* ಸಂಗ್ರಹಣಾ ಲೇಯರ್, ಇದು ಮೊದಲ ಅಕ್ಷದ ಮೇಲೆ ಈ ಟೆನ್ಸರ್‌ನ ಸರಾಸರಿಯನ್ನು ತೆಗೆದುಕೊಳ್ಳುತ್ತದೆ, ಅಂದರೆ ವಿಭಿನ್ನ ಪದಗಳಿಗೆ ಹೊಂದಿರುವ ಎಲ್ಲಾ $n$ ಇನ್‌ಪುಟ್ ಟೆನ್ಸರ್‌ಗಳ ಸರಾಸರಿಯನ್ನು ಲೆಕ್ಕಹಾಕುತ್ತದೆ. ಈ ಲೇಯರ್ ಅನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸಲು, ನಾವು `Lambda` ಲೇಯರ್ ಅನ್ನು ಬಳಸುತ್ತೇವೆ ಮತ್ತು ಅದಕ್ಕೆ ಸರಾಸರಿ ಲೆಕ್ಕಹಾಕುವ ಫಂಕ್ಷನ್ ಅನ್ನು ಪಾಸ್ ಮಾಡುತ್ತೇವೆ. ಔಟ್‌ಪುಟ್ 100 ಆಕಾರದಿರುತ್ತದೆ ಮತ್ತು ಇದು ಸಂಪೂರ್ಣ ಇನ್‌ಪುಟ್ ಕ್ರಮದ ಸಂಖ್ಯಾತ್ಮಕ ಪ್ರತಿನಿಧನೆ ಆಗಿರುತ್ತದೆ.
* ಅಂತಿಮ `Dense` ರೇಖೀಯ ವರ್ಗೀಕರಣ.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` ಮುದ್ರಣದಲ್ಲಿ, **output shape** ಕಾಲಮ್‌ನಲ್ಲಿ, ಮೊದಲ ಟೆನ್ಸರ್ ಆಯಾಮ `None` ಮಿನಿಬ್ಯಾಚ್ ಗಾತ್ರಕ್ಕೆ ಹೊಂದಿಕೆಯಾಗಿದ್ದು, ಎರಡನೇ ಆಯಾಮ ಟೋಕನ್ ಸರಣಿಯ ಉದ್ದಕ್ಕೆ ಹೊಂದಿಕೆಯಾಗುತ್ತದೆ. ಮಿನಿಬ್ಯಾಚ್‌ನಲ್ಲಿರುವ ಎಲ್ಲಾ ಟೋಕನ್ ಸರಣಿಗಳ ಉದ್ದಗಳು ವಿಭಿನ್ನವಾಗಿರುತ್ತವೆ. ಇದನ್ನು ಹೇಗೆ ನಿರ್ವಹಿಸುವುದು ಎಂಬುದನ್ನು ಮುಂದಿನ ವಿಭಾಗದಲ್ಲಿ ಚರ್ಚಿಸುವೆವು.

ಈಗ ನೆಟ್‌ವರ್ಕ್ ಅನ್ನು ತರಬೇತಿಮಾಡೋಣ:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **ಗಮನಿಸಿ** ನಾವು ಡೇಟಾದ ಉಪಸಮೂಹದ ಆಧಾರದ ಮೇಲೆ ವೆಕ್ಟರೈಜರ್ ಅನ್ನು ನಿರ್ಮಿಸುತ್ತಿದ್ದೇವೆ. ಪ್ರಕ್ರಿಯೆಯನ್ನು ವೇಗಗೊಳಿಸಲು ಇದನ್ನು ಮಾಡಲಾಗುತ್ತದೆ, ಮತ್ತು ಇದರಿಂದ ನಮ್ಮ ಪಠ್ಯದಲ್ಲಿನ ಎಲ್ಲಾ ಟೋಕನ್‌ಗಳು ಶಬ್ದಕೋಶದಲ್ಲಿ ಇರದ ಪರಿಸ್ಥಿತಿ ಉಂಟಾಗಬಹುದು. ಈ ಸಂದರ್ಭದಲ್ಲಿ, ಆ ಟೋಕನ್‌ಗಳನ್ನು ನಿರ್ಲಕ್ಷಿಸಲಾಗುತ್ತದೆ, ಇದರಿಂದ ಸ್ವಲ್ಪ ಕಡಿಮೆ ನಿಖರತೆ ಉಂಟಾಗಬಹುದು. ಆದಾಗ್ಯೂ, ನಿಜ ಜೀವನದಲ್ಲಿ ಪಠ್ಯದ ಉಪಸಮೂಹವು ಸಾಮಾನ್ಯವಾಗಿ ಉತ್ತಮ ಶಬ್ದಕೋಶ ಅಂದಾಜು ನೀಡುತ್ತದೆ.


### ಬದಲಾಗುವ ಕ್ರಮದ ಗಾತ್ರಗಳನ್ನು ನಿರ್ವಹಿಸುವುದು

ಮಿನಿಬ್ಯಾಚ್‌ಗಳಲ್ಲಿ ತರಬೇತಿ ಹೇಗೆ ನಡೆಯುತ್ತದೆ ಎಂಬುದನ್ನು ತಿಳಿದುಕೊಳ್ಳೋಣ. ಮೇಲಿನ ಉದಾಹರಣೆಯಲ್ಲಿ, ಇನ್‌ಪುಟ್ ಟೆನ್ಸರ್‌ನ ಆಯಾಮ 1 ಆಗಿದ್ದು, ನಾವು 128-ದೀರ್ಘ ಮಿನಿಬ್ಯಾಚ್‌ಗಳನ್ನು ಬಳಸುತ್ತೇವೆ, ಆದ್ದರಿಂದ ಟೆನ್ಸರ್‌ನ ನಿಜವಾದ ಗಾತ್ರ $128 \times 1$ ಆಗಿದೆ. ಆದರೆ, ಪ್ರತಿ ವಾಕ್ಯದ ಟೋಕನ್‌ಗಳ ಸಂಖ್ಯೆ ವಿಭಿನ್ನವಾಗಿರುತ್ತದೆ. ನಾವು `TextVectorization` ಲೇಯರ್ ಅನ್ನು ಒಬ್ಬೊಬ್ಬ ಇನ್‌ಪುಟ್‌ಗೆ ಅನ್ವಯಿಸಿದರೆ, ಟೋಕನ್‌ಗಳ ಸಂಖ್ಯೆ ವಿಭಿನ್ನವಾಗಿರುತ್ತದೆ, ಅದು ಪಠ್ಯವನ್ನು ಹೇಗೆ ಟೋಕನೈಸ್ ಮಾಡಲಾಗಿದೆ ಎಂಬುದರ ಮೇಲೆ ಅವಲಂಬಿತವಾಗಿರುತ್ತದೆ:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


ಆದರೆ, ನಾವು ವೆಕ್ಟರೈಜರ್ ಅನ್ನು ಹಲವಾರು ಕ್ರಮಗಳಿಗೆ ಅನ್ವಯಿಸಿದಾಗ, ಅದು ಆಯತಾಕಾರದ ಟೆನ್ಸರ್ ಅನ್ನು ಉತ್ಪಾದಿಸಬೇಕು, ಆದ್ದರಿಂದ ಅದು ಬಳಸದ ಅಂಶಗಳನ್ನು PAD ಟೋಕನ್ (ನಮ್ಮ ಪ್ರಕರಣದಲ್ಲಿ ಶೂನ್ಯ) ಮೂಲಕ ತುಂಬುತ್ತದೆ:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

ಇಲ್ಲಿ ನಾವು ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ನೋಡಬಹುದು:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **ಗಮನಿಸಿ**: ಪ್ಯಾಡಿಂಗ್ ಪ್ರಮಾಣವನ್ನು ಕಡಿಮೆ ಮಾಡಲು, ಕೆಲವು ಸಂದರ್ಭಗಳಲ್ಲಿ ಡೇಟಾಸೆಟ್‌ನ ಎಲ್ಲಾ ಕ್ರಮಗಳನ್ನು ಹೆಚ್ಚುತ್ತಿರುವ ಉದ್ದದ ಕ್ರಮದಲ್ಲಿ (ಅಥವಾ, ಹೆಚ್ಚು ನಿಖರವಾಗಿ, ಟೋಕನ್‌ಗಳ ಸಂಖ್ಯೆಯ ಕ್ರಮದಲ್ಲಿ) ವಿಂಗಡಿಸುವುದು ಅರ್ಥಪೂರ್ಣವಾಗುತ್ತದೆ. ಇದರಿಂದ ಪ್ರತಿ ಮಿನಿಬ್ಯಾಚ್ ಸಮಾನ ಉದ್ದದ ಕ್ರಮಗಳನ್ನು ಹೊಂದಿರುತ್ತದೆ ಎಂದು ಖಚಿತವಾಗುತ್ತದೆ.


## ಅರ್ಥಪೂರ್ಣ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳು: Word2Vec

ನಮ್ಮ ಹಿಂದಿನ ಉದಾಹರಣೆಯಲ್ಲಿ, ಎम्बೆಡ್ಡಿಂಗ್ ಲೇಯರ್ ಪದಗಳನ್ನು ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧನೆಗಳಿಗೆ ನಕ್ಷೆ ಮಾಡಿಕೊಳ್ಳಲು ಕಲಿತು, ಆದರೆ ಈ ಪ್ರತಿನಿಧನೆಗಳಿಗೆ ಅರ್ಥಪೂರ್ಣ ಅರ್ಥ ಇರಲಿಲ್ಲ. ಸಮಾನಾರ್ಥಕ ಪದಗಳು ಅಥವಾ ಸಮಾನಾರ್ಥಕ ಪದಗಳು ಕೆಲವು ವೆಕ್ಟರ್ ದೂರ (ಉದಾಹರಣೆಗೆ ಯೂಕ್ಲಿಡಿಯನ್ ದೂರ) ಅರ್ಥದಲ್ಲಿ ಪರಸ್ಪರ ಹತ್ತಿರ ಇರುವ ವೆಕ್ಟರ್‌ಗಳಿಗೆ ಹೊಂದಿಕೊಳ್ಳುವಂತೆ ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧನೆಯನ್ನು ಕಲಿಯುವುದು ಒಳ್ಳೆಯದು.

ಅದಕ್ಕಾಗಿ, ನಾವು Word2Vec ಎಂಬ ತಂತ್ರವನ್ನು ಬಳಸಿಕೊಂಡು ದೊಡ್ಡ ಪಠ್ಯ ಸಂಗ್ರಹದ ಮೇಲೆ ನಮ್ಮ ಎम्बೆಡ್ಡಿಂಗ್ ಮಾದರಿಯನ್ನು ಪೂರ್ವಪ್ರಶಿಕ್ಷಣ ಮಾಡಬೇಕಾಗುತ್ತದೆ. ಇದು ಪದಗಳ ವಿತರಿತ ಪ್ರತಿನಿಧನೆಯನ್ನು ಉತ್ಪಾದಿಸಲು ಬಳಸುವ ಎರಡು ಪ್ರಮುಖ ವಾಸ್ತುಶಿಲ್ಪಗಳ ಮೇಲೆ ಆಧಾರಿತವಾಗಿದೆ:

 - **ನಿರಂತರ ಬ್ಯಾಗ್-ಆಫ್-ವರ್ಡ್ಸ್** (CBoW), ಇಲ್ಲಿ ನಾವು ಸುತ್ತಲೂ ಇರುವ ಸನ್ನಿವೇಶದಿಂದ ಒಂದು ಪದವನ್ನು ಊಹಿಸಲು ಮಾದರಿಯನ್ನು ತರಬೇತುಗೊಳಿಸುತ್ತೇವೆ. n-ಗ್ರಾಮ್ $(W_{-2},W_{-1},W_0,W_1,W_2)$ ನೀಡಿದಾಗ, ಮಾದರಿಯ ಗುರಿ $(W_{-2},W_{-1},W_1,W_2)$ ನಿಂದ $W_0$ ಅನ್ನು ಊಹಿಸುವುದು.
 - **ನಿರಂತರ ಸ್ಕಿಪ್-ಗ್ರಾಮ್** CBoW ಗೆ ವಿರುದ್ಧವಾಗಿದೆ. ಮಾದರಿ ಪ್ರಸ್ತುತ ಪದವನ್ನು ಊಹಿಸಲು ಸುತ್ತಲಿನ ಸನ್ನಿವೇಶ ಪದಗಳ ವಿಂಡೋವನ್ನು ಬಳಸುತ್ತದೆ.

CBoW ವೇಗವಾಗಿ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ, ಮತ್ತು ಸ್ಕಿಪ್-ಗ್ರಾಮ್ ನಿಧಾನವಾಗಿದ್ದರೂ ಅಪರೂಪದ ಪದಗಳನ್ನು ಪ್ರತಿನಿಧಿಸುವಲ್ಲಿ ಉತ್ತಮ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ.

![ಪದಗಳನ್ನು ವೆಕ್ಟರ್‌ಗಳಿಗೆ ಪರಿವರ್ತಿಸಲು CBoW ಮತ್ತು ಸ್ಕಿಪ್-ಗ್ರಾಮ್ ಆಲ್ಗಾರಿಥಮ್‌ಗಳನ್ನು ತೋರಿಸುವ ಚಿತ್ರ.](../../../../../translated_images/kn/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News ಡೇಟಾಸೆಟ್‌ನಲ್ಲಿ ಪೂರ್ವಪ್ರಶಿಕ್ಷಿತ Word2Vec ಎम्बೆಡ್ಡಿಂಗ್‌ನೊಂದಿಗೆ ಪ್ರಯೋಗ ಮಾಡಲು, ನಾವು **gensim** ಗ್ರಂಥಾಲಯವನ್ನು ಬಳಸಬಹುದು. ಕೆಳಗೆ 'neural' ಗೆ ಅತ್ಯಂತ ಸಮಾನವಾದ ಪದಗಳನ್ನು ಕಂಡುಹಿಡಿಯಲಾಗಿದೆ.

> **ಗಮನಿಸಿ:** ನೀವು ಮೊದಲ ಬಾರಿಗೆ ಪದ ವೆಕ್ಟರ್‌ಗಳನ್ನು ರಚಿಸುವಾಗ, ಅವುಗಳನ್ನು ಡೌನ್‌ಲೋಡ್ ಮಾಡಲು ಸ್ವಲ್ಪ ಸಮಯ ಬೇಕಾಗಬಹುದು!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


ನಾವು ಶಬ್ದದಿಂದ ವೆಕ್ಟರ್ ಎಂಬೆಡ್ಡಿಂಗ್ ಅನ್ನು ಕೂಡ ತೆಗೆದುಹಾಕಬಹುದು, ಇದನ್ನು ವರ್ಗೀಕರಣ ಮಾದರಿಯನ್ನು ತರಬೇತಿಗೆ ಬಳಸಬಹುದು. ಎಂಬೆಡ್ಡಿಂಗ್‌ನಲ್ಲಿ 300 ಘಟಕಗಳಿವೆ, ಆದರೆ ಇಲ್ಲಿ ಸ್ಪಷ್ಟತೆಗಾಗಿ ವೆಕ್ಟರ್‌ನ ಮೊದಲ 20 ಘಟಕಗಳನ್ನು ಮಾತ್ರ ತೋರಿಸಲಾಗಿದೆ:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

ಸಾಮಾಂತಿಕ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳ ಮಹತ್ವದ ವಿಷಯವೆಂದರೆ ನೀವು ಅರ್ಥಾಧಾರಿತವಾಗಿ ವೆಕ್ಟರ್ ಎನ್‌ಕೋಡಿಂಗ್ ಅನ್ನು ನಿಯಂತ್ರಿಸಬಹುದು. ಉದಾಹರಣೆಗೆ, ನಾವು *king* ಮತ್ತು *woman* ಎಂಬ ಪದಗಳ ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಾನದಷ್ಟು ಹತ್ತಿರವಾಗಿರುವ ಮತ್ತು *man* ಎಂಬ ಪದದಿಂದ ಸಾಧ್ಯವಾದಷ್ಟು ದೂರವಾಗಿರುವ ಪದವನ್ನು ಹುಡುಕಲು ಕೇಳಬಹುದು:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

ಮೇಲಿನ ಉದಾಹರಣೆಯಲ್ಲಿ ಕೆಲವು ಆಂತರಿಕ GenSym ಮಾಯಾಜಾಲವನ್ನು ಬಳಸಲಾಗಿದೆ, ಆದರೆ ಮೂಲ ತರ್ಕವು ನಿಜವಾಗಿಯೂ ಸರಳವಾಗಿದೆ. ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳ ಬಗ್ಗೆ ಒಂದು ಆಸಕ್ತಿದಾಯಕ ವಿಷಯವೆಂದರೆ ನೀವು ಎम्बೆಡ್ಡಿಂಗ್ ವೆಕ್ಟರ್‌ಗಳ ಮೇಲೆ ಸಾಮಾನ್ಯ ವೆಕ್ಟರ್ ಕಾರ್ಯಾಚರಣೆಗಳನ್ನು ನಡೆಸಬಹುದು, ಮತ್ತು ಅದು ಪದಗಳ **ಅರ್ಥಗಳ** ಮೇಲೆ ಕಾರ್ಯಾಚರಣೆಗಳನ್ನು ಪ್ರತಿಬಿಂಬಿಸುತ್ತದೆ. ಮೇಲಿನ ಉದಾಹರಣೆಯನ್ನು ವೆಕ್ಟರ್ ಕಾರ್ಯಾಚರಣೆಗಳ ರೂಪದಲ್ಲಿ ವ್ಯಕ್ತಪಡಿಸಬಹುದು: ನಾವು **KING-MAN+WOMAN** ಗೆ ಹೊಂದಿಕೆಯಾಗುವ ವೆಕ್ಟರ್ ಅನ್ನು ಲೆಕ್ಕಹಾಕುತ್ತೇವೆ (ಕಾರ್ಯಾಚರಣೆಗಳು `+` ಮತ್ತು `-` ಸಂಬಂಧಿತ ಪದಗಳ ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಿಗಳ ಮೇಲೆ ನಡೆಸಲಾಗುತ್ತವೆ), ಮತ್ತು ನಂತರ ಆ ವೆಕ್ಟರ್‌ಗೆ ನಿಕಟವಾದ ಪದವನ್ನು ನಿಘಂಟಿನಲ್ಲಿ ಹುಡುಕುತ್ತೇವೆ:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: ನಾವು *man* ಮತ್ತು *woman* ವೆಕ್ಟರ್‌ಗಳಿಗೆ ಸಣ್ಣ ಗುಣಾಂಕಗಳನ್ನು ಸೇರಿಸಬೇಕಾಯಿತು - ಅವುಗಳನ್ನು ತೆಗೆದುಹಾಕಿ ಏನಾಗುತ್ತದೆ ಎಂದು ಪ್ರಯತ್ನಿಸಿ.

ಅತ್ಯಂತ ಸಮೀಪದ ವೆಕ್ಟರ್ ಅನ್ನು ಕಂಡುಹಿಡಿಯಲು, ನಾವು TensorFlow ಯಂತ್ರವನ್ನು ಬಳಸಿಕೊಂಡು ನಮ್ಮ ವೆಕ್ಟರ್ ಮತ್ತು ಶಬ್ದಕೋಶದಲ್ಲಿರುವ ಎಲ್ಲಾ ವೆಕ್ಟರ್‌ಗಳ ನಡುವಿನ ದೂರಗಳ ವೆಕ್ಟರ್ ಅನ್ನು ಲೆಕ್ಕಹಾಕುತ್ತೇವೆ, ನಂತರ `argmin` ಬಳಸಿ ಕನಿಷ್ಠ ಶಬ್ದದ ಸೂಚ್ಯಂಕವನ್ನು ಹುಡುಕುತ್ತೇವೆ.


Word2Vec ಪದಾರ್ಥಗಳ ಅರ್ಥವನ್ನು ವ್ಯಕ್ತಪಡಿಸುವ ಉತ್ತಮ ವಿಧಾನವಾಗಿದ್ದರೂ, ಇದಕ್ಕೆ ಹಲವಾರು ದುರ್ಬಲತೆಗಳಿವೆ, ಅವುಗಳಲ್ಲಿ ಪ್ರಮುಖವಾದವುಗಳು:

* CBoW ಮತ್ತು skip-gram ಮಾದರಿಗಳು ಎರಡೂ **ಭವಿಷ್ಯವಾಣಿ ಆಧಾರಿತ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳು**, ಮತ್ತು ಅವುಗಳು ಕೇವಲ ಸ್ಥಳೀಯ ಸಂದರ್ಭವನ್ನು ಮಾತ್ರ ಪರಿಗಣಿಸುತ್ತವೆ. Word2Vec ಜಾಗತಿಕ ಸಂದರ್ಭವನ್ನು ಉಪಯೋಗಿಸುವುದಿಲ್ಲ.
* Word2Vec ಪದದ **ರೂಪಶಾಸ್ತ್ರ**ವನ್ನು ಪರಿಗಣಿಸುವುದಿಲ್ಲ, ಅಂದರೆ ಪದದ ಅರ್ಥವು ಪದದ ವಿವಿಧ ಭಾಗಗಳ ಮೇಲೆ, ಉದಾಹರಣೆಗೆ ಮೂಲ ಪದದ ಮೇಲೆ ಅವಲಂಬಿತವಾಗಿರಬಹುದು ಎಂಬ ಸಂಗತಿಯನ್ನು ಗಮನದಲ್ಲಿಟ್ಟುಕೊಳ್ಳುವುದಿಲ್ಲ.

**FastText** ಎರಡನೇ ಮಿತಿಯನ್ನು ಮೀರಿ ಹೋಗಲು ಪ್ರಯತ್ನಿಸುತ್ತದೆ, ಮತ್ತು Word2Vec ಮೇಲೆ ನಿರ್ಮಿತವಾಗಿದ್ದು, ಪ್ರತಿ ಪದ ಮತ್ತು ಅದರೊಳಗಿನ ಅಕ್ಷರ n-ಗ್ರಾಮ್‌ಗಳಿಗಾಗಿ ವೆಕ್ಟರ್ ಪ್ರತಿನಿಧಾನಗಳನ್ನು ಕಲಿಯುತ್ತದೆ. ಪ್ರತಿನಿಧಾನದ ಮೌಲ್ಯಗಳನ್ನು ಪ್ರತಿ ತರಬೇತಿ ಹಂತದಲ್ಲಿ ಸರಾಸರಿ ಮಾಡಿ ಒಂದು ವೆಕ್ಟರ್ ಆಗಿ ರೂಪಿಸುತ್ತದೆ. ಇದರಿಂದ ಪೂರ್ವಪ್ರಶಿಕ್ಷಣದಲ್ಲಿ ಹೆಚ್ಚಿನ ಗಣನೆ ಅಗತ್ಯವಿದ್ದರೂ, ಪದ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳು ಉಪಪದ ಮಾಹಿತಿಯನ್ನು ಸಂಕೇತಗೊಳಿಸಲು ಸಾಧ್ಯವಾಗುತ್ತದೆ.

ಮತ್ತೊಂದು ವಿಧಾನ, **GloVe**, ಪದ-ಸಂದರ್ಭ ಮ್ಯಾಟ್ರಿಕ್ಸ್‌ನ ಘಟಕೀಕರಣದ ಆಧಾರದ ಮೇಲೆ ಪದ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳಿಗೆ ವಿಭಿನ್ನ ವಿಧಾನವನ್ನು ಬಳಸುತ್ತದೆ. ಮೊದಲು, ಇದು ವಿವಿಧ ಸಂದರ್ಭಗಳಲ್ಲಿ ಪದಗಳ ಸಂಭವಗಳ ಸಂಖ್ಯೆಯನ್ನು ಎಣಿಸುವ ದೊಡ್ಡ ಮ್ಯಾಟ್ರಿಕ್ಸ್ ಅನ್ನು ನಿರ್ಮಿಸುತ್ತದೆ, ನಂತರ ಈ ಮ್ಯಾಟ್ರಿಕ್ಸ್ ಅನ್ನು ಕಡಿಮೆ ಆಯಾಮಗಳಲ್ಲಿ ಪ್ರತಿನಿಧಿಸಲು ಪ್ರಯತ್ನಿಸುತ್ತದೆ, ಪುನರ್ ನಿರ್ಮಾಣ ನಷ್ಟವನ್ನು ಕನಿಷ್ಠಗೊಳಿಸುವ ರೀತಿಯಲ್ಲಿ.

gensim ಗ್ರಂಥಾಲಯವು ಆ ಪದ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ಬೆಂಬಲಿಸುತ್ತದೆ, ಮತ್ತು ಮೇಲಿನ ಮಾದರಿ ಲೋಡ್ ಮಾಡುವ ಕೋಡ್ ಅನ್ನು ಬದಲಾಯಿಸುವ ಮೂಲಕ ನೀವು ಅವುಗಳೊಂದಿಗೆ ಪ್ರಯೋಗ ಮಾಡಬಹುದು.


## Keras ನಲ್ಲಿ pretrained embeddings ಬಳಸುವುದು

ಮೇಲಿನ ಉದಾಹರಣೆಯನ್ನು ನಾವು semantic embeddings, ಉದಾಹರಣೆಗೆ Word2Vec, ಬಳಸಿ ನಮ್ಮ embedding ಲೇಯರ್‌ನ ಮ್ಯಾಟ್ರಿಕ್ಸ್ ಅನ್ನು ಪೂರ್ವನಿರ್ಧರಿಸಲು ಬದಲಾಯಿಸಬಹುದು. pretrained embedding ಮತ್ತು ಪಠ್ಯ ಕಾರ್ಪಸ್‌ನ ಶಬ್ದಕೋಶಗಳು ಸಾಮಾನ್ಯವಾಗಿಯೂ ಹೊಂದಿಕೆಯಾಗುವುದಿಲ್ಲ, ಆದ್ದರಿಂದ ನಾವು ಒಂದನ್ನು ಆಯ್ಕೆ ಮಾಡಬೇಕಾಗುತ್ತದೆ. ಇಲ್ಲಿ ನಾವು ಎರಡು ಸಾಧ್ಯವಾದ ಆಯ್ಕೆಗಳನ್ನು ಪರಿಶೀಲಿಸುತ್ತೇವೆ: tokenizer ಶಬ್ದಕೋಶವನ್ನು ಬಳಸುವುದು ಮತ್ತು Word2Vec embeddings ನಿಂದ ಶಬ್ದಕೋಶವನ್ನು ಬಳಸುವುದು.

### tokenizer ಶಬ್ದಕೋಶವನ್ನು ಬಳಸುವುದು

tokenizer ಶಬ್ದಕೋಶವನ್ನು ಬಳಸುವಾಗ, ಶಬ್ದಕೋಶದ ಕೆಲವು ಪದಗಳಿಗೆ Word2Vec embeddings ಇರುತ್ತವೆ, ಮತ್ತು ಕೆಲವು ಇಲ್ಲದಿರಬಹುದು. ನಮ್ಮ ಶಬ್ದಕೋಶದ ಗಾತ್ರ `vocab_size` ಆಗಿದ್ದು, Word2Vec embedding ವೆಕ್ಟರ್ ಉದ್ದ `embed_size` ಆಗಿದ್ದರೆ, embedding ಲೇಯರ್ ಅನ್ನು `vocab_size`$\times$`embed_size` ಆಕಾರದ ತೂಕ ಮ್ಯಾಟ್ರಿಕ್ಸ್ ಮೂಲಕ ಪ್ರತಿನಿಧಿಸಲಾಗುತ್ತದೆ. ನಾವು ಈ ಮ್ಯಾಟ್ರಿಕ್ಸ್ ಅನ್ನು ಶಬ್ದಕೋಶದ ಮೂಲಕ ಹೋಗಿ ತುಂಬಿಸುತ್ತೇವೆ:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Word2Vec ಶಬ್ದಕೋಶದಲ್ಲಿ ಇಲ್ಲದ ಪದಗಳಿಗಾಗಿ, ನಾವು ಅವುಗಳನ್ನು ಶೂನ್ಯಗಳಾಗಿ ಬಿಡಬಹುದು ಅಥವಾ ಯಾದೃಚ್ಛಿಕ ವೆಕ್ಟರ್ ಅನ್ನು ರಚಿಸಬಹುದು.

ಈಗ ನಾವು ಪೂರ್ವಪ್ರಶಿಕ್ಷಿತ ತೂಕಗಳೊಂದಿಗೆ embedding ಲೇಯರ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸಬಹುದು:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

ಈಗ ನಮ್ಮ ಮಾದರಿಯನ್ನು ತರಬೇತುಗೊಳಿಸೋಣ.


In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **ಗಮನಿಸಿ**: ನಾವು `Embedding` ಅನ್ನು ಸೃಷ್ಟಿಸುವಾಗ `trainable=False` ಅನ್ನು ಹೊಂದಿಸಿದ್ದೇವೆ, ಅಂದರೆ ನಾವು Embedding ಲೇಯರ್ ಅನ್ನು ಮರುಶಿಕ್ಷಣ ಮಾಡುತ್ತಿಲ್ಲ. ಇದರಿಂದ ನಿಖರತೆ ಸ್ವಲ್ಪ ಕಡಿಮೆಯಾಗಬಹುದು, ಆದರೆ ತರಬೇತಿಯನ್ನು ವೇಗಗೊಳಿಸುತ್ತದೆ.

### embedding ಶಬ್ದಕೋಶವನ್ನು ಬಳಸುವುದು

ಹಿಂದಿನ ವಿಧಾನದಲ್ಲಿ ಒಂದು ಸಮಸ್ಯೆ ಎಂದರೆ TextVectorization ಮತ್ತು Embedding ನಲ್ಲಿ ಬಳಸಿದ ಶಬ್ದಕೋಶಗಳು ವಿಭಿನ್ನವಾಗಿವೆ. ಈ ಸಮಸ್ಯೆಯನ್ನು ದಾಟಿಸಲು, ನಾವು ಕೆಳಗಿನ ಪರಿಹಾರಗಳಲ್ಲಿ ಒಂದನ್ನು ಬಳಸಬಹುದು:
* ನಮ್ಮ ಶಬ್ದಕೋಶದ ಮೇಲೆ Word2Vec ಮಾದರಿಯನ್ನು ಮರುಶಿಕ್ಷಣ ಮಾಡುವುದು.
* pretrained Word2Vec ಮಾದರಿಯಿಂದ ಶಬ್ದಕೋಶವನ್ನು ಪಡೆದು ನಮ್ಮ ಡೇಟಾಸೆಟ್ ಅನ್ನು ಲೋಡ್ ಮಾಡುವುದು. ಡೇಟಾಸೆಟ್ ಲೋಡ್ ಮಾಡುವಾಗ ಬಳಸುವ ಶಬ್ದಕೋಶವನ್ನು ನಿರ್ದಿಷ್ಟಪಡಿಸಬಹುದು.

ಇದರಲ್ಲಿ ಎರಡನೇ ವಿಧಾನ ಸುಲಭವಾಗಿರುವಂತೆ ತೋರುತ್ತದೆ, ಆದ್ದರಿಂದ ಅದನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸೋಣ. ಮೊದಲು, Word2Vec embeddings ನಿಂದ ತೆಗೆದುಕೊಂಡ ನಿರ್ದಿಷ್ಟ ಶಬ್ದಕೋಶದೊಂದಿಗೆ `TextVectorization` ಲೇಯರ್ ಅನ್ನು ಸೃಷ್ಟಿಸುತ್ತೇವೆ:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

gensim ಶಬ್ದ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳ ಗ್ರಂಥಾಲಯದಲ್ಲಿ, `get_keras_embeddings` ಎಂಬ ಅನುಕೂಲಕರ ಫಂಕ್ಷನ್ ಇದೆ, ಇದು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ನಿಮಗಾಗಿ ಸಂಬಂಧಿತ Keras ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳ ಲೇಯರ್ ಅನ್ನು ರಚಿಸುತ್ತದೆ.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


ನಾವು ಹೆಚ್ಚಿನ ನಿಖರತೆಯನ್ನು ಕಾಣದಿರುವ ಒಂದು ಕಾರಣವೆಂದರೆ ನಮ್ಮ ಡೇಟಾಸೆಟ್‌ನ ಕೆಲವು ಪದಗಳು ಪೂರ್ವಪ್ರಶಿಕ್ಷಿತ GloVe ಶಬ್ದಕೋಶದಲ್ಲಿ ಇಲ್ಲದಿರುವುದರಿಂದ ಅವುಗಳನ್ನು ಮೂಲತಃ ನಿರ್ಲಕ್ಷಿಸಲಾಗುತ್ತದೆ. ಇದನ್ನು ಮೀರಿ ಹೋಗಲು, ನಾವು ನಮ್ಮ ಡೇಟಾಸೆಟ್ ಆಧರಿಸಿ ನಮ್ಮದೇ embeddings ಅನ್ನು ತರಬೇತಿಮಾಡಬಹುದು.


## ಸಾಂದರ್ಭಿಕ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳು

ಪಾರಂಪರಿಕ ಪೂರ್ವಪ್ರಶಿಕ್ಷಿತ ಎम्बೆಡ್ಡಿಂಗ್ ಪ್ರತಿನಿಧಿಗಳಾದ Word2Vec ಮುಂತಾದವುಗಳ ಪ್ರಮುಖ ಮಿತಿ ಎಂದರೆ, ಅವು ಒಂದು ಪದದ ಕೆಲವು ಅರ್ಥಗಳನ್ನು ಹಿಡಿದಿಟ್ಟುಕೊಳ್ಳಬಹುದು ಆದರೂ, ವಿಭಿನ್ನ ಅರ್ಥಗಳನ್ನು ವಿಭಿನ್ನವಾಗಿ ಗುರುತಿಸಲು ಸಾಧ್ಯವಿಲ್ಲ. ಇದರಿಂದ ನಂತರದ ಮಾದರಿಗಳಲ್ಲಿ ಸಮಸ್ಯೆಗಳು ಉಂಟಾಗಬಹುದು.

ಉದಾಹರಣೆಗೆ, 'play' ಎಂಬ ಪದಕ್ಕೆ ಈ ಎರಡು ವಿಭಿನ್ನ ವಾಕ್ಯಗಳಲ್ಲಿ ವಿಭಿನ್ನ ಅರ್ಥಗಳಿವೆ:
- ನಾನು ನಾಟಕಾಲಯದಲ್ಲಿ ಒಂದು **play** ನೋಡಲು ಹೋದೆನು.
- ಜಾನ್ ತನ್ನ ಸ್ನೇಹಿತರೊಂದಿಗೆ **play** ಮಾಡಲು ಬಯಸುತ್ತಾನೆ.

ನಾವು ಚರ್ಚಿಸಿದ ಪೂರ್ವಪ್ರಶಿಕ್ಷಿತ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳು 'play' ಪದದ ಎರಡೂ ಅರ್ಥಗಳನ್ನು ಒಂದೇ ಎम्बೆಡ್ಡಿಂಗ್‌ನಲ್ಲಿ ಪ್ರತಿನಿಧಿಸುತ್ತವೆ. ಈ ಮಿತಿಯನ್ನು ಮೀರುವುದುಗಾಗಿ, ನಾವು **ಭಾಷಾ ಮಾದರಿ** ಆಧಾರಿತ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳನ್ನು ನಿರ್ಮಿಸಬೇಕಾಗುತ್ತದೆ, ಇದು ದೊಡ್ಡ ಪಠ್ಯ ಸಂಗ್ರಹದ ಮೇಲೆ ತರಬೇತಿಗೊಳಿಸಲಾಗಿದೆ ಮತ್ತು ಪದಗಳನ್ನು ವಿಭಿನ್ನ ಸಾಂದರ್ಭಿಕಗಳಲ್ಲಿ ಹೇಗೆ ಜೋಡಿಸಬಹುದು ಎಂಬುದನ್ನು *ಬಲ್ಲದು*. ಸಾಂದರ್ಭಿಕ ಎम्बೆಡ್ಡಿಂಗ್‌ಗಳ ಬಗ್ಗೆ ಚರ್ಚಿಸುವುದು ಈ ಪಾಠದ ವ್ಯಾಪ್ತಿಗೆ ಸೇರಿಲ್ಲ, ಆದರೆ ಮುಂದಿನ ಘಟಕದಲ್ಲಿ ಭಾಷಾ ಮಾದರಿಗಳ ಬಗ್ಗೆ ಮಾತನಾಡುವಾಗ ನಾವು ಅವುಗಳತ್ತ ಮರಳುತ್ತೇವೆ.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ತಪ್ಪುಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
